In [1]:
import glob
import pandas as pd
from pathlib import Path
import pickle

In [2]:
from PIL import Image
from tqdm import tqdm

In [3]:
#  dataframe multi-thread
from pandarallel import pandarallel
# Initialization
pandarallel.initialize()
# Standard pandas apply
# Parallel apply
# df.parallel_apply(func, progress_bar=True)
# progress_bar=True

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
with open('/nas/chenyi/datasets_cls/map/gallery/stem2path_gallery2mw202208.pickle', 'rb') as f:
    stem2path = pickle.load(f)

In [5]:
gall_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_csv/gall2m_df20220725.csv')
# gall_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_csv/gall2m_hw_df20220725.csv', index=False)

In [26]:
width_df = pd.DataFrame()
width_df['stem'] = stem2width.keys()
width_df['width'] = stem2width.values()
height_df = pd.DataFrame()
height_df['stem'] = stem2height.keys()
height_df['height'] = stem2height.values()

In [27]:
path_df = pd.DataFrame()
path_df['stem'] = stem2path.keys()
path_df['path'] = stem2path.values()

In [28]:
cross_df = path_df[~path_df['stem'].isin(width_df['stem'])]

In [29]:
width_df.tail(1)

,stem,width
2391033,ffffb9d9-d2a5-11ec-820f-5cfb3aa6f5e6,1279


In [30]:
height_df.tail(1)

,stem,height
2391033,ffffb9d9-d2a5-11ec-820f-5cfb3aa6f5e6,1706


In [6]:
def get_hw(p):
    img = Image.open(p)
    w, h = img.size
    # w_list.append(w)
    # h_list.append(h)
    return w,h

In [8]:
test_df = gall_df

In [7]:
# %%time
# w_list, h_list = [], []
# w_list, h_list = zip(*test_df['path'].apply(lambda x: get_hw(x) ))
# test_df['width'] = w_list
# test_df['height'] = h_list

In [12]:
%%time
w_list, h_list = zip(*test_df['oss_path'].parallel_apply(lambda x: get_hw(x)))
test_df['width'] = w_list
test_df['height'] = h_list

CPU times: user 1.94 s, sys: 2.04 s, total: 3.98 s
Wall time: 19min 28s


In [13]:
test_df.to_csv('/nas/chenyi/datasets_cls/map/gallery/gall2m_df.csv', index=False)

In [21]:
stay_dict = {}
map_df = test_df.loc[:, ['stem', 'width']]
for k, v in tqdm(zip(map_df[map_df.columns[0]].values, test_df[map_df.columns[1]].values)):
    if k not in stay_dict.keys():
        stay_dict[k] = v

 ... (more hidden) ...


In [22]:
with open('/nas/chenyi/datasets_cls/map/gallery/stem2width_gallery2m20220728.pickle', 'wb') as f:
    pickle.dump(stay_dict, f)

In [23]:
stay_dict = {}
map_df = test_df.loc[:, ['stem', 'height']]
for k, v in tqdm(zip(map_df[map_df.columns[0]].values, test_df[map_df.columns[1]].values)):
    if k not in stay_dict.keys():
        stay_dict[k] = v

 ... (more hidden) ...


In [24]:
with open('/nas/chenyi/datasets_cls/map/gallery/stem2height_gallery2m20220728.pickle', 'wb') as f:
    pickle.dump(stay_dict, f)

In [103]:
width_df = pd.DataFrame()
width_df['stem'] = stem2width.keys()
width_df['width'] = stem2width.values()

#### 生成gallery目标df

In [ ]:
with open('/nas/chenyi/datasets_cls/map/gallery/stem2width_gallery2m202208.pickle', 'rb') as f:
    stem2width = pickle.load(f)
with open('/nas/chenyi/datasets_cls/map/gallery/stem2height_gallery2m202208.pickle', 'rb') as f:
    stem2height = pickle.load(f)

In [27]:
# gall_df['width'] = gall_df['stem'].map(stem2width)

In [126]:
gall_df['height'] = gall_df['stem'].map(stay_dict)